In [15]:
from io import StringIO
from scipy.io import mmread
from scipy.io import mminfo
from google.colab import drive
drive.mount("/content/drive")
datAddr = "/content/drive/MyDrive/CMSC818J_Test/"
from scipy.sparse import csr_matrix
from scipy import sparse
import numpy as np
import sys

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Read the matrix in mtx format and convert it to csr format 
m = mmread(datAddr+'spaceShuttleEntry_3.mtx')
mat_csr = m.tocsr()
mat_csrB = m.tocsc()

## Creating a small example to test on

In [2]:
# row_mat = 4
# cols_mat= 4

# mat1 = [[0 for _ in range(cols_mat)] for _ in range(row_mat)]
# editing the individual elements
# mat1[0][0], mat1[0][1], mat1[0][2], mat1[0][3] = 0,0,8,0
# mat1[1][0], mat1[1][1], mat1[1][2], mat1[1][3] = 7,9,6,0
# mat1[2][0], mat1[2][1], mat1[2][2], mat1[2][3] = 0,5,0,4
# mat1[3][0], mat1[3][1], mat1[3][2], mat1[3][3] = 3,0,0,2
# print(f'Matrix is {mat1}')
# mat_csr = sparse.csr_matrix(mat1)
# mat_csrB = sparse.csc_matrix(mat1)

Matrix is [[0, 0, 8, 0], [7, 9, 6, 0], [0, 5, 0, 4], [3, 0, 0, 2]]


## Baseline Architecture 
 

## Assumptions
1. Each multiplication and accumulation takes 3 cycles
2. Reading the rows from memory takes 2 cycles
3. Storing the result of multiplication and accumulation takes 1 cycle. NO merging required since we are implementing inner product. 
4. PEs write back the output in parallel and can read the input in parallel

In [24]:
# ENUMS for PE_stats
NUMBER_OF_CYCLES = 1 
MAX_NUM_NON_ZEROS = 2
MIN_NUM_NON_ZEROS = 3

total_PEs= 2 # number of PEs available. Can change this number as required
PE_stats = []
for i in range(total_PEs):
  PE_stat = dict()
  PE_stat[NUMBER_OF_CYCLES] = 0 
  PE_stat[MAX_NUM_NON_ZEROS] = 0 
  PE_stat[MIN_NUM_NON_ZEROS ] = sys.maxsize
  PE_stats.append(PE_stat)
PE_stats

[{1: 0, 2: 0, 3: 9223372036854775807}, {1: 0, 2: 0, 3: 9223372036854775807}]

In [25]:
# Simulates PE MAC operations
def PE_mult(output_row_num, output_col_num):
  PE_to_process = PE_lst.pop()
  accumulation = 0
  # optimization for inner product: whichever dictionary has smallest non-zeros, we iterate over that. 
  if len(B_dict) < len(PE_to_process):
    for col,data in B_dict.items():
      if col in PE_to_process:
        # multiply and then accumulate  
        accumulation += PE_to_process[col] * data 
        PE_stats[PE_num][NUMBER_OF_CYCLES] += 3
    output_mat2[output_row_num][output_col_num] = accumulation 
    PE_stats[PE_num][NUMBER_OF_CYCLES] += 1
  else:
    for col,data in PE_to_process.items():
      if col in B_dict:
        # multiply and then accumulate  
        accumulation += B_dict[col] * data 
        PE_stats[PE_num][NUMBER_OF_CYCLES] += 3 

    output_mat2[output_row_num][output_col_num] = accumulation
    # Update PE stats
    PE_stats[PE_num][NUMBER_OF_CYCLES] += 1
    
  if len(PE_to_process) > PE_stats[PE_num][MAX_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MAX_NUM_NON_ZEROS] = len(PE_to_process)
  elif len(PE_to_process) < PE_stats[PE_num][MIN_NUM_NON_ZEROS]: 
      PE_stats[PE_num][MIN_NUM_NON_ZEROS] = len(PE_to_process)
      



In [26]:
mat_csrB_len = len(mat_csrB.indptr) - 1
mat_csrA_len = len(mat_csr.indptr) - 1
output_mat_len = len(mat_csrB.indptr) - 1
PE_num = 0

# acts as the buffer that stores rows to be allocated to PEs
PE_lst = []
# matrix to store the file output 
output_mat2 = [[0 for _ in range(output_mat_len)] for _ in range(output_mat_len)]
data_index_B = 0 
row_index_B = 0
pointer_index_B = 0 
output_col_index = 0
output_row_index = 0


for j in range(mat_csrB_len): 
  # Decode col of matrix B from csc format
  numOfElems_B = (mat_csrB.indptr[pointer_index_B + 1] - mat_csrB.indptr[pointer_index_B])
  data_row_B = mat_csrB.data[data_index_B:numOfElems_B+data_index_B]
  col_B = mat_csrB.indices[row_index_B:numOfElems_B+data_index_B]

  # The decoded cols and corresponding data values are zipped into a dictionary 
  # data structure to easily pattern match with rows when computing inner product
  B_dict = dict(zip(col_B, data_row_B))
  data_index_B += numOfElems_B
  row_index_B += numOfElems_B
  pointer_index_B += 1
  data_index_A = 0
  col_index_A = 0
  pointer_index_A = 0
  output_row_index = 0
  for i in range(mat_csrA_len):
    # Decode rows of matrix A from csr format 
    numOfElems = (mat_csr.indptr[pointer_index_A + 1] - mat_csr.indptr[pointer_index_A])
    data_row_A = mat_csr.data[data_index_A:numOfElems+data_index_A]
    row_A = mat_csr.indices[col_index_A:numOfElems+data_index_A]
    PE_lst.append(dict(zip(row_A, data_row_A)))
    data_index_A += numOfElems
    col_index_A += numOfElems
    pointer_index_A += 1
    # allocate it to a PE using round-robin strategy 
    PE_mult(output_row_index, output_col_index)
    PE_num = (PE_num + 1) % total_PEs 
    output_row_index += 1
  output_col_index += 1 



In [27]:
output_mat2
output_mat2_reshape = np.reshape(output_mat2, (output_mat_len, output_mat_len))
correct_output = np.dot(m.todense(),m.todense())
# Validates if matrix multiplication is done correctly
print(np.allclose(output_mat2_reshape, correct_output))
# prints the number of cycles per PE, max and min number of zeros per PE
print(PE_stats)

True
[{1: 6647135, 2: 30, 3: 2}, {1: 6749126, 2: 1700, 3: 2}]


## Ideas
1. Just have one PE list each of the PE's working on it can just pop. Can 
also have another list that stores row and col values that correponds to the 
PE list. 
2. For multiplication, for each col of matrix B we multiply it with all the rows of matrix A https://www.mathsisfun.com/algebra/matrix-multiplying.html 
3. So matrix B should be stored in csc format. 
4. The PE_lst is the buffer that stores all the computation that need to be done. We need to find a way for the PE to be assigned the next computation as soon as it is done. For that, we can pre-compute how many cycles the multiplication would take. 
5. For each PE, we need to store the idle cycles at the end. If a PE keeps getting assigned smaller non-zeros then it would stay idle more at the end. 
6. Have counters to measure load-imbalance 
7. Create a function that simulates reading from memory